In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.datasets import imdb
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')
nltk.download('averaged_perceptron_tagger')

In [ ]:
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_df.head()

In [ ]:
train_df = train_df[['text','target']].copy()
train_df.info()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
7552/7613

In [ ]:
train_df = train_df[['text','target']].copy()
train_df.info()

In [ ]:
test_df = test_df[['id','text']].copy()
test_df.info()

# Text Cleaning

### 1) lower

In [ ]:
train_df["text_clean"] = train_df["text"].apply(lambda x: x.lower())
train_df.head()

In [ ]:
test_df["text_clean"] = test_df["text"].apply(lambda x: x.lower())
test_df.head()

### 2) remove URL, HTML, emojis

In [ ]:
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_URL(x))
test_df["text_clean"] = test_df["text_clean"].apply(lambda x: remove_URL(x))

In [ ]:
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_html(x))
test_df["text_clean"] = test_df["text_clean"].apply(lambda x: remove_html(x))

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_emojis(x))
test_df["text_clean"] = test_df["text_clean"].apply(lambda x: remove_emojis(x))

In [ ]:
def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_punct(x))
test_df["text_clean"] = test_df["text_clean"].apply(lambda x: remove_punct(x))
train_df.head()

In [ ]:
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7f]',r'', text)
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_non_ascii(x))
test_df["text_clean"] = test_df["text_clean"].apply(lambda x: remove_non_ascii(x))
train_df.head()

# Preprocessing

In [ ]:
train_df['tokenized'] = train_df['text_clean'].apply(word_tokenize)
train_df.head()

In [ ]:
test_df['tokenized'] = test_df['text_clean'].apply(word_tokenize)
test_df.head()

In [ ]:
stop = set(stopwords.words('english'))
train_df['stopwords_removed'] = train_df['tokenized'].apply(lambda x: [word for word in x if word not in stop])
train_df.head()

In [ ]:
test_df['stopwords_removed'] = test_df['tokenized'].apply(lambda x: [word for word in x if word not in stop])
test_df.head()

In [ ]:
def porter_stemmer(text):
    stemmer = nltk.PorterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [ ]:
train_df['porter_stemmer'] = train_df['stopwords_removed'].apply(lambda x: porter_stemmer(x))
train_df.head()

In [ ]:
test_df['porter_stemmer'] = test_df['stopwords_removed'].apply(lambda x: porter_stemmer(x))
test_df.head()

In [ ]:
def lemmatize_word(text):
    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word) for word in text]
    return lemma
train_df['lemmatize'] = train_df['stopwords_removed'].apply(lambda x: lemmatize_word(x))
train_df.head()

In [ ]:
test_df['lemmatize'] = test_df['stopwords_removed'].apply(lambda x: lemmatize_word(x))
test_df.head()

In [ ]:
train_df['final_text']=train_df['lemmatize'].apply(lambda x: ''.join(i+' ' for i in x))
test_df['final_text']=test_df['lemmatize'].apply(lambda x: ''.join(i+' ' for i in x))
train_df.head()

# Count-freq

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (1,1))
x_tr = vectorizer.fit_transform(train_df['final_text'])
vectorizer.get_feature_names_out()

In [ ]:
x_ts = vectorizer.transform(test_df['final_text'])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_tr.toarray(), train_df['target'], test_size=0.2, random_state=None)

In [ ]:
x_tr.shape

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(16270,)))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
from keras import losses
from keras import metrics
from tensorflow.keras import optimizers

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs=20,
                    batch_size=16,
                    validation_data=(x_val, y_val))

In [ ]:
history_dict = history.history
history_dict.keys()


acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## 2) Tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfvectorizer = TfidfVectorizer(ngram_range = (1,2))
x_tr = tfvectorizer.fit_transform(train_df['final_text'])
tfvectorizer.get_feature_names_out()
x_ts = tfvectorizer.transform(test_df['final_text'])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_tr.toarray(), train_df['target'], test_size=0.2, random_state=None)

In [ ]:
x_train.shape

In [ ]:
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(63245,)))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_val, y_val))

In [ ]:
history_dict = history.history
history_dict.keys()


acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
sub = model.predict(x_ts.toarray())>0.5

In [ ]:
submission_df = {"id":test_df['id'],
                 "target":sub.T[0].astype(int)}
submission = pd.DataFrame(submission_df)
submission.to_csv('submission.csv',index=False)